In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Rengarajan\\My_Projects\\DiseasePrognosis\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Rengarajan\\My_Projects\\DiseasePrognosis'

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("artifacts\data_ingestion\Disease_pred_dataset\data.csv")
df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [7]:
df.shape

(5004, 133)

In [8]:
df.isnull().sum()

itching                 0
skin_rash               0
nodal_skin_eruptions    0
continuous_sneezing     0
shivering               0
                       ..
inflammatory_nails      0
blister                 0
red_sore_around_nose    0
yellow_crust_ooze       0
prognosis               0
Length: 133, dtype: int64

In [9]:
# this code will get all columns & dataypes and save it in schema.yaml
# this code not necessary while building pipelines
import yaml

# Get column names and data types
columns_data_types = df.dtypes.to_dict()

# Convert the data types to string representation
columns_data_types = {str(column): str(dtype) for column, dtype in columns_data_types.items()}

# Save to a YAML file
with open('schema.yaml', 'w') as file:
    yaml.dump(columns_data_types, file, default_flow_style=False)

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    status_file: str
    all_schema: dict

In [11]:
from diseaseprognosis.constants import *
from diseaseprognosis.utils import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.status_file,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [13]:
import os
from diseaseprognosis import logger

In [14]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [15]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-11-16 15:44:57,812: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-16 15:44:57,812: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-16 15:44:57,838: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-16 15:44:57,838: INFO: common: created directory at: artifacts]
[2023-11-16 15:44:57,846: INFO: common: created directory at: artifacts/data_validation]
